In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import cv, XGBRegressor

In [2]:
path = "../data/brca/tcga/processed/GSE176078/ct_minor/prop_estimate_scores_primary_tumor_unstranded_subset_CID3586.csv"
scores = pd.read_csv(path,sep=',')
scores.head()

,ID,B cells Memory,B cells Naive,T cells CD8+,T cells CD4+,NK cells,NKT cells,Monocyte,Stromal_score,Immune_score,ESTIMATE_score
1,3C-AAAU,-5.431209,0.0000,0.0,0.0,-4.910174,-0.132676,-2.184786,-711.41,-906.98,-1618.39
2,3C-AALI,-7.368048,-5.5144,0.0,0.0,0.000000,-0.243500,-1.553714,122.66,511.24,633.91
3,3C-AALJ,0.000000,0.0000,0.0,0.0,-5.933138,-0.196054,-1.740791,400.91,539.25,940.16
4,3C-AALK,-3.926095,0.0000,0.0,0.0,-3.966422,-0.190500,-2.004020,799.11,-3.53,795.58
5,4H-AAAK,-5.534338,0.0000,0.0,0.0,0.000000,-0.170725,-1.877333,1075.36,-46.96,1028.40


In [3]:
scores.dtypes

ID                 object
B cells Memory    float64
B cells Naive     float64
T cells CD8+      float64
T cells CD4+      float64
NK cells          float64
NKT cells         float64
Monocyte          float64
Stromal_score     float64
Immune_score      float64
ESTIMATE_score    float64
dtype: object

In [7]:
scores.columns.tolist()

['ID',
 'B cells Memory',
 'B cells Naive',
 'T cells CD8+',
 'T cells CD4+',
 'NK cells',
 'NKT cells',
 'Monocyte',
 'Stromal_score',
 'Immune_score',
 'ESTIMATE_score']

In [11]:
cols = [
 'B cells Memory',
 'B cells Naive',
 'T cells CD8+',
 'T cells CD4+',
 'NK cells',
 'NKT cells',
 'Monocyte',
]
data = scores[cols]
labels = scores["Immune_score"]

In [12]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)


In [16]:

# Create an XGBoost regressor
model = XGBRegressor(objective='reg:squarederror')

# Perform cross-validation on the training data
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')

# Print the mean squared error for each fold
print("Cross-validation scores:\n", scores)


Cross-validation scores:
 [0.65640141 0.63515417 0.70419807 0.66327054 0.57122769]


In [20]:

# Train the XGBoost regressor on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the mean squared error on the test data
r2 = r2_score(y_test, y_pred)
print("R2 test data:", r2)

R2 test data: 0.6472198863314771


In [21]:
# Extract variable importance
var_importance = model.get_booster().get_score(importance_type='weight')

# Extract feature weights
weights = model.get_booster().get_score(importance_type='total_gain')
total_gain = sum(weights.values())
weights = {k:v/total_gain for k,v in weights.items()}

# Extract list of columns
columns = model.get_booster().feature_names

# Print the results
print("Variable importance:\n", var_importance)
print("Feature weights:\n", weights)
print("List of columns:\n", columns)


Variable importance:
 {'B cells Memory': 869.0, 'B cells Naive': 68.0, 'T cells CD8+': 37.0, 'T cells CD4+': 21.0, 'NK cells': 638.0, 'NKT cells': 880.0, 'Monocyte': 685.0}
Feature weights:
 {'B cells Memory': 0.0630980042661303, 'B cells Naive': 0.006093377828000967, 'T cells CD8+': 0.006877750815326864, 'T cells CD4+': 0.0020138481342993663, 'NK cells': 0.13613274184876634, 'NKT cells': 0.18505231061494834, 'Monocyte': 0.6007319664925278}
List of columns:
 ['B cells Memory', 'B cells Naive', 'T cells CD8+', 'T cells CD4+', 'NK cells', 'NKT cells', 'Monocyte']


In [23]:
model.get_booster().get_score(importance_type='total_gain')

{'B cells Memory': 68826816.0,
 'B cells Naive': 6646609.5,
 'T cells CD8+': 7502197.5,
 'T cells CD4+': 2196690.0,
 'NK cells': 148492544.0,
 'NKT cells': 201853632.0,
 'Monocyte': 655273792.0}

In [24]:
from sklearn.inspection import permutation_importance
# Perform permutation feature importance on the test set
result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)

# Extract the p-values for each feature
p_values = result.importances_mean / result.importances_std

# Print the results
print("P-values for each feature:\n", p_values)


P-values for each feature:
 [ 1.74988505  1.10285663 -0.36438166  0.15952988  9.25100099  5.66558653
  8.75178388]


In [25]:
result

{'importances_mean': array([ 3.09979269e-02,  4.57285292e-03, -2.29573476e-04,  1.31874240e-04,
         3.11388199e-01,  2.29484640e-01,  6.96448909e-01]),
 'importances_std': array([0.01771426, 0.00414637, 0.00063004, 0.00082664, 0.03365995,
        0.04050501, 0.07957794]),
 'importances': array([[ 2.00727658e-02,  6.52419077e-02,  2.63415375e-02,
          2.92688479e-02,  1.41942742e-04,  3.27595297e-02,
          4.83976323e-02,  1.02805318e-02,  3.60614625e-02,
          4.14131108e-02],
        [-3.46669615e-03,  1.54934647e-03,  3.06879324e-04,
          6.54919378e-03,  3.87278804e-03,  7.02682379e-03,
          4.72508845e-03,  1.18646910e-02,  8.61318957e-03,
          4.68722494e-03],
        [-2.46134684e-04,  1.89406638e-04, -2.35986764e-04,
         -1.45499500e-03, -1.42630139e-04, -3.11228119e-04,
         -7.01349478e-04,  1.19936146e-03, -2.49026026e-04,
         -3.43152651e-04],
        [ 6.13727530e-04, -8.51701216e-04, -7.28444434e-04,
          1.53192720e-04, 